In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
257,AFG,Asia,Afghanistan,2020-09-13,38641.0,35.0,45.286,1420.0,0.0,1.429,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
446,ALB,Europe,Albania,2020-09-13,11185.0,164.0,154.714,330.0,3.0,2.571,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
704,DZA,Africa,Algeria,2020-09-13,48007.0,255.0,276.571,1605.0,6.0,8.000,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-12,France,371965.0,0.0,30926.0,0.0,0.0,0.0
0,2020-09-12,China,90171.0,0.0,4738.0,0.0,0.0,0.0
0,2020-09-12,Italy,286196.0,0.0,35625.0,0.0,0.0,0.0
0,2020-09-12,Spain,575512.0,0.0,29811.0,0.0,0.0,0.0
0,2020-09-12,United States,6518771.0,0.0,194097.0,0.0,0.0,0.0
0,2020-09-12,World,28947593.0,0.0,924226.0,0.0,0.0,0.0
0,2020-09-12,United Kingdom,363960.0,0.0,41659.0,0.0,0.0,0.0
0,2020-09-12,Germany,259807.0,0.0,9354.0,0.0,0.0,0.0
0,2020-09-12,Iran,401524.0,0.0,23154.0,0.0,0.0,0.0
0,2020-09-12,Turkey,289340.0,0.0,6997.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-13,France,371965.0,373911.0,30926.0,30910.0,-1946.0,16.0
0,2020-09-13,China,90171.0,90168.0,4738.0,4734.0,3.0,4.0
0,2020-09-13,Italy,286196.0,286297.0,35625.0,35603.0,-101.0,22.0
0,2020-09-12,Spain,575512.0,566326.0,29811.0,29747.0,9186.0,64.0
0,2020-09-13,United States,6518771.0,6486108.0,194097.0,193701.0,32663.0,396.0
0,2020-09-13,World,28947593.0,28802775.0,924226.0,920931.0,144818.0,3295.0
0,2020-09-13,United Kingdom,363960.0,365174.0,41659.0,41623.0,-1214.0,36.0
0,2020-09-13,Germany,259807.0,259428.0,9354.0,9349.0,379.0,5.0
0,2020-09-13,Iran,401524.0,399940.0,23154.0,23029.0,1584.0,125.0
0,2020-09-13,Turkey,289340.0,289635.0,6997.0,6999.0,-295.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")